In [23]:
nbIndividus = 100 # nombre d'individus dans le graphe

# taille des clusters du graphe (min/max)
foyer = (2, 5)
communaute = (4, 5) # nb de foyers dans une communauté

# application
joursNotif = 14

pDetection = 0.9
pSignalement = 0.5
pQNotif = 0.5 # proba de mise en confinement lors de la réception d'une notif

# probas initialisation
rencontreFoyer = 0.9 # proba de rencontrer un autre membre du foyer
rencontreCommunaute = 0.01 # proba de rencontrer un autre membre de la communauté
rencontreNational = 0.001

# probas
pContamination = 0.5 # proba de contaminer un autre individu

pAtoG = 0.1 # proba de passer de asymptotique à guéri
pAtoIS = 0.1 # passage de asymptotique à avec symptomes
pAtoD = 0.1 # proba de décès

pIStoG = 0.1 # avec symptomes à gueri
pIStoD = 0.1

pQSymptomes = 0.2 # proba confinement lors de détection symptomes

facteurConfinement = 3 # Réduction des probas de rencontre lors du confinement
jourConfinement = 14

In [24]:
# Librairies
import random

In [25]:
SAIN=0
MALADEASYMP=1
MALADE=2
GUERI=3
DECEDE=4

In [46]:
# Initialisation du graphe
individus = [{"state": SAIN, "confine": False, "resteConfinement": 0, "appli": True} for individu in range(nbIndividus)]
foyers = []
communautes = []

adj = [[] for l in range(nbIndividus)]
adj[0].append({"noeud": 3, "proba": 0.2})

# Fonctions de gestion du graphe

In [27]:
rencontres = [[[] for jour in range(joursNotif)] for individu in range(nbIndividus)]

In [41]:
def contamination(i, j):
    if individus[i]['state'] == SAIN and individus[j]['state'] == SAIN:
        return
    if individus[i]['state'] >= GUERI or individus[j]['state'] >= GUERI:
        return # on ne peut pas contaminer les individus guéris ou décédés
    
    if indivus[i] == SAIN:
        contamination(j, i)
        
        return
    
    if random.random() > pContamination:
        return # pas contamination

    individus[j]['state'] = MALADEASYMP

In [48]:
# Passage au jour suivant du graphe
def step():
    for rencontre in rencontres:
        rencontre.append([]) # contiendra les nouvelles rencontres du jour

    # On constate toutes les rencontres entre individus
    for i in range(nbIndividus):
        for arete in adj[i]:
            j = arete['noeud']
            if j < i:
                continue # on ne regarde qu'un sens de chaque arête
            
            facteur = 1
            if individus[i]['resteConfinement'] > 0:
                facteur *= facteurConfinement
            if individus[j]['resteConfinement'] > 0:
                facteur *= facteurConfinement
                
            if random.random() > arete['proba'] / facteur: # Si i et/ou j sont confinés, réduction de leur proba de rencontre
                continue # pas de rencontre
        
            # Si i et j ont l'appli, on note la rencontre
            if individus[i].appli and individus[j]['appli'] and random.random() < pDetection: 
                rencontres[i][-1].append(j) 
                rencontres[j][-1].append(i)
            
            contamination(i, j)
    
    # On passe au jour suivant
    for i in range(nbIndividus):
        individus[i]['resteConfinement'] -= 1

    for i, individu in enumerate(individus): # On met à jour les états des individus
        if individu['state'] == MALADEASYMP:
            action = random.random()
            if action < pAtoG:
                individu['state'] = GUERI
            elif action > 1 - pAtoIS:
                individu['state'] = MALADE
                
                # envoi des notifs (rencontres[i] vide si i n'a pas l'appli)
                for jourRencontre in rencontres[i]:
                    for contact in jourRencontre:
                        if random.random() < pQNotif:
                            individus[contact]['resteConfinement'] = jourConfinement
                
                if random.random() < pQSymptomes: # mise en confinement à la détection des symptomes
                    individu['resteConfinement'] = jourConfinement
                
        elif individu['state'] == MALADE:
            action = random.random()
            if action < pIStoG:
                individu['state'] = GUERI
            elif action > 1 - pIStoD:
                individu['state'] = DECES
    
    # On supprime le plus vieux jour de l'historique
    for rencontre in rencontres:
        rencontre.pop(0)
        
            

In [49]:
# Affichage
step()